<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# NeurIPS - Run ZSMMT

Goal : train models on ZSMMT

Data : ZSMMT

Models : SelfAttnLNP and ConvLNP and ConvCNP 

Loss : NLLLloss

Runs : 1

In [1]:
import os

os.chdir("..")

In [2]:
import submitit

from train_imgs import main, parse_arguments
from results import get_exp_args

class Run:
    def checkpoint(self, args):
        """Resubmits the same callable with the same arguments but makes sure continnue from last chckpnt."""
        args.is_continue_train = True
        return submitit.utils.DelayedSubmission(self, args)

    def __call__(self, args):
        job_env = submitit.utils.JobEnvironment()
        args.starting_run = args.starting_run * job_env.num_tasks + job_env.local_rank
        return main(args)


In [3]:
args = get_exp_args("exp_allmm_hetero_lb01", is_load=False, is_reeval=True)
len(args)

20

In [4]:
args = [a for a in args if  a.model == "SelfAttnNPF"]

In [5]:
len(args)

10

In [6]:
#args = [a for a in args if a.model == "ConvNPF"]
#for a in args:
#    a.batch_size = 16

In [7]:
executor=submitit.SlurmExecutor(folder="logs/%j", max_num_timeout=3)
executor.update_parameters(num_gpus=1, 
                           time=60*24*2,  
                           cpus_per_task=10, 
                           mem='32GB',
                           constraint="volta32gb",
                           partition="dev"
                          )

In [8]:
jobs = executor.map_array(Run(), args)

In [9]:
############################################################

In [11]:
jobs

[SlurmJob<job_id=27822537_0, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27822537_1, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27822537_2, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27822537_3, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27822537_4, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27822537_5, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27822537_6, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27822537_7, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27822537_8, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27822537_9, task_id=0, state="COMPLETED">]

In [24]:
for job in jobs:
    print("--------------------------------")
    print(job.stdout())

--------------------------------
submitit INFO (2020-07-01 02:08:03,987) - Starting with JobEnvironment(job_id=27810797_0, hostname=learnfair0345, local_rank=0(1), node=0(1), global_rank=0(1))
submitit INFO (2020-07-01 02:08:03,987) - Loading pickle: /private/home/yannd/projects/NPF/logs/27810797_0/27810797_0_submitted.pkl

--- Training zsmms/ConvNPFL_NllLNPF/run_0 ---

  epoch    train_loss    valid_loss    cp        dur
-------  ------------  ------------  ----  ---------
      1     -532.3872     -773.0313     +  2335.6907
      2     -975.3403     -774.3262     +  2335.2012
      3    -1072.8983    -1199.3321     +  2335.6802
      4    -1123.6847    -1202.0103     +  2334.2772
      5    -1210.4947    -1302.9684     +  2335.1615
      6    -1204.3884    -1359.1878     +  2335.0533
      7    -1241.5686    -1202.7957        2335.5857
      8    -1218.6903    -1319.1519        2335.7262
      9    -1269.3268    -1399.2686     +  2335.4668
     10    -1292.1913    -1450.5893     +  2

In [17]:
for j in jobs:
    print("--------------------------------")
    print(j.stderr())

--------------------------------
INFO:utils.data.imgs:Resizing ZeroShotMultiMNIST ...
INFO:utils.data.imgs:Resizing ZeroShotMultiMNIST ...
INFO:submitit:Job completed successfully

--------------------------------
INFO:utils.data.imgs:Resizing ZeroShotMultiMNIST ...
INFO:utils.data.imgs:Resizing ZeroShotMultiMNIST ...
submitit ERROR (2020-06-27 18:44:40,889) - Submitted job triggered an exception
ERROR:submitit:Submitted job triggered an exception
Traceback (most recent call last):
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/_submit.py", line 6, in <module>
    submitit_main()
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/submission.py", line 18

In [3]:
for j in jobs:
    j.cancel()

NameError: name 'jobs' is not defined